Similar to end of 2020-06-14 , but I want to try many iterations, still batching to try avoid 
kernel exploding

In [1]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1

In [22]:
datadir = '/opt/data'
localdir = '/opt/program'

# This time only take about a 1/60th  of the data... 0.017% ... 
tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv'
                     ).sample(frac=0.017, random_state=42)
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)


In [24]:
X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# preproc
(X_transformed,
     one_hot_enc, le,
     y_enc) = pv1.preprocess(X_train, y_train, # X[train_index]
                         neighborhoods)
labels = le.classes_

# Test set
X_test_transformed = one_hot_enc.transform(X_test)
y_test_enc = le.transform(y_test)

In [28]:

size = X_transformed.shape[0]
print(size)
indices = np.random.choice(range(size), size=size, replace=False)

parts = fu.get_partitions(indices, slice_size=1000)
len(parts[0])
len(Counter(y_enc[parts[0]]))

10753


54

In [29]:
y_test_enc.shape

(3585,)

In [30]:
len(parts)

11

In [ ]:
# This time... going to just perhaps, shrink the test set and only test after every iteration..

In [ ]:
workdir = fu.make_work_dir(); print(workdir)
fu.log(workdir, 'Starting')
prev_model = None
loss_vec = []; acc_vec = []

clf = xgb.XGBClassifier()

for epoch in tqdm(range(100), desc='epochs'):
    fu.log(workdir, f'[{epoch}:{i}] Done fit')
    for i, part in enumerate(tqdm(parts, leave=False, desc='batches')):
        clf.fit(X_transformed[part], y_enc[part], xgb_model=prev_model)
        fu.log(workdir, f'[{epoch}:{i}] Done fit')

        prev_model = f'{workdir}/model.xg'
        clf.save_model(prev_model)

    y_prob_vec = clf.predict_proba(X_test_transformed)
    fu.log(workdir, f'[{epoch}] Done predict_proba')

    loss = fu.big_logloss(y_test_enc, y_prob_vec, labels)
    fu.log(workdir, f'[{epoch}] Done big_logloss, loss={loss}.')

    loss_vec.append(loss)

    acc = accuracy_score(y_test_enc, np.argmax(y_prob_vec, axis=1))
    acc_vec.append(acc)
    fu.log(workdir, f'[{epoch}] Done accuracy, acc={acc}.')
    
    

/opt/program/artifacts/2020-06-17T042025Z
